In [1]:
import pandas as pd
import numpy as np 

# STEP1. CLEAN SALES DATA (SINGLE QUARTER) #

In [2]:
# Read one file at a time into df, name df by 's'+'issue number ''
s136 = "Files/Sales/Issue-136-Sales-tables-March-2021-quarter.xlsx"
s136 = pd.read_excel(s136, sheet_name="Postcode", na_values='-', header=6)

In [3]:
# Rename columns
rename_cols= {'Postcode':'postcode', 
             'Dwelling Type':'dwelling_type', 
             "First Quartile Sales Price\n$'000s" : '25%_price',
             "Median Sales Price\n$'000s" : 'median_price', 
             "Third Quartile Sales Price\n'000s" : '75%_price',
             "Mean Sales Price\n$'000s" : 'mean_price',
             'Sales\nNo.':'sales_no',
             'Qtly change in Median':'Qdelta_median_price',
             'Annual change in Median':'Adelta_median_price',
             'Qtly change in Count':'Qdelta_sales_no',
             'Annual change in Count':'Adelta_sales_no'}
s136.rename(columns=rename_cols, inplace=True)

# Drop unwanted columns
s136 = s136.drop(columns=['25%_price', '75%_price'], axis=1)

s136.head(5)

,postcode,dwelling_type,median_price,mean_price,sales_no,Qdelta_median_price,Adelta_median_price,Qdelta_sales_no,Adelta_sales_no
0,2000,Total,1371.0,2794.0,184,0.1425,0.0711,0.0888,0.5862
1,2000,Strata,1371.0,2794.0,184,0.1331,0.0632,0.1018,0.6429
2,2007,Total,763.0,754.0,s,0.1713,0.0235,0.0000,0.3000
3,2007,Non Strata,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007,Strata,710.0,688.0,s,0.0906,-0.0470,-0.0769,0.2000


In [4]:
print("number of postcodes:", s136['postcode'].nunique())
print("number of rows:", s136.shape[0],'\n')
print(s136.isnull().sum())

number of postcodes: 544
number of rows: 1427 

postcode                 0
dwelling_type            0
median_price           316
mean_price             316
sales_no               316
Qdelta_median_price    316
Adelta_median_price    321
Qdelta_sales_no        316
Adelta_sales_no        321
dtype: int64


<b>IMPUTATION</b>

Roughly 1/4 of the data contains null values, which comes from:
<em>"for confidentiality, we don't report sales in any geographical area where the number of sales is 10 or less (shown as na).</em> Also: <em>"Statistics calculated from sample sizes between 10 an 30 are shown by an 's' in the relevant table".</em> This will be the primary rules of imputation:
* **For 'sales_no':**
    * Replace na with 5
    * Replace s with 20
    
* **For 'median_price' and 'mean_price'**
    * Fill na with the median of price of the same dwelling type

In [5]:
# Imputation of sales number

# na
s136.loc[s136['sales_no'].isnull(), 'sales_no'] = 5.0
s136['sales_no'].isnull().any()

# s
s136.loc[s136['sales_no'] == 's', 'sales_no'] = 20.0
s136['sales_no'] = s136['sales_no'].astype(float) # Cast type as float

print("number of na in sales_no:", s136['sales_no'].isnull().sum())
print("data type of sales_no:", s136['sales_no'].dtype)

number of na in sales_no: 0
data type of sales_no: float64


In [6]:
# Imputation of median_price (by dwelling type)

# Total
imp_total = s136.loc[(s136['median_price'].notna()) & (s136['dwelling_type']=='Total'),
                     'median_price'].median() # calculate imputer value 
    
s136.loc[(s136['median_price'].isnull()) & (s136['dwelling_type']=='Total'),
         'median_price']= imp_total #impute

# Strata 
imp_strata = s136.loc[(s136['median_price'].notna()) & (s136['dwelling_type']=='Strata'),
                     'median_price'].median() 
    
s136.loc[(s136['median_price'].isnull()) & (s136['dwelling_type']=='Strata'),
         'median_price']= imp_strata 

# Non-Strata
imp_ns = s136.loc[(s136['median_price'].notna()) & (s136['dwelling_type']=='Non Strata'),
                     'median_price'].median() 
    
s136.loc[(s136['median_price'].isnull()) & (s136['dwelling_type']=='Non Strata'),
         'median_price']= imp_ns

print("number of na in median_price:", s136['median_price'].isnull().sum())

number of na in median_price: 0


In [7]:
# Imputation of mean_price (by dwelling type)

# Total
imp_total = s136.loc[(s136['mean_price'].notna()) & (s136['dwelling_type']=='Total'),
                     'mean_price'].median() # calculate imputer value 
    
s136.loc[(s136['mean_price'].isnull()) & (s136['dwelling_type']=='Total'),
         'mean_price']= imp_total #impute

# Strata 
imp_strata = s136.loc[(s136['mean_price'].notna()) & (s136['dwelling_type']=='Strata'),
                     'mean_price'].median() 
    
s136.loc[(s136['mean_price'].isnull()) & (s136['dwelling_type']=='Strata'),
         'mean_price']= imp_strata 

# Non-Strata
imp_ns = s136.loc[(s136['mean_price'].notna()) & (s136['dwelling_type']=='Non Strata'),
                  'mean_price'].median() 
    
s136.loc[(s136['mean_price'].isnull()) & (s136['dwelling_type']=='Non Strata'),
         'mean_price']= imp_ns

print("number of na in mean_price:", s136['mean_price'].isnull().sum())

number of na in mean_price: 0


In [8]:
s136.describe().round(2)

,postcode,median_price,mean_price,sales_no,Qdelta_median_price,Adelta_median_price,Qdelta_sales_no,Adelta_sales_no
count,1427.00,1427.00,1427.00,1427.00,1111.00,1106.00,1111.00,1106.00
mean,2378.20,923.04,994.20,51.47,0.06,0.13,-0.03,0.62
std,256.21,628.89,706.76,58.94,0.14,0.20,0.41,0.85
min,2000.00,120.00,125.00,5.00,-0.90,-0.46,-0.59,-0.49
25%,2148.00,610.00,635.50,20.00,-0.01,0.03,-0.22,0.17
50%,2330.00,760.50,801.50,20.00,0.05,0.11,-0.10,0.41
75%,2575.00,960.00,1072.50,68.00,0.12,0.20,0.06,0.74
max,2880.00,6400.00,7374.00,397.00,0.78,2.04,6.50,7.00


**ADD TIME PERIOD TAG**

In [9]:
s136['key_s'] = 's136'
s136['time_period'] = '2021 Q1'
s136['year'] = '2021'
s136['quarter'] = '1'

print(s136.shape)

(1427, 13)


### End of the sales data cleaning process ###
----

# STEP2. CLEAN RENT DATA (SINGLE QUARTER) #


In [10]:
# Read into df
r135 = "Files/Rent/Issue-135-Rent-tables-March-2021-quarter.xlsx"
r135 = pd.read_excel(r135, sheet_name="Postcode", na_values='-', header=7)

# Drop unwanted columns
r135 = r135.drop(columns=['First Quartile Weekly Rent for New Bonds\n$',
                          'Third Quartile Weekly Rent for New Bonds\n$'],
                axis=1)

# Rename columns
rename_cols= {'Postcode':'postcode',
              'Dwelling Types':'dwelling_type', 
              'Number of Bedrooms':'bed_number',
              'Median Weekly Rent for New Bonds\n$': 'median_rent_newb',
              'New Bonds Lodged\nNo.' : 'new_bonds_no',
              'Total Bonds Held\nNo.': 'total_bonds_no',
              'Quarterly change in Median Weekly Rent':'Qdelta_median_rent',
              'Annual change in Median Weekly Rent':'Adelta_median_rent',
              'Quarterly change in New Bonds Lodged':'Qdelta_new_bonds',
              'Annual change in New Bonds Lodged':'Adelta_new_bonds'}
r135.rename(columns=rename_cols,inplace=True)

r135.head(10)

,postcode,dwelling_type,bed_number,median_rent_newb,new_bonds_no,total_bonds_no,Qdelta_median_rent,Adelta_median_rent,Qdelta_new_bonds,Adelta_new_bonds
0,2000,Total,Total,600.0,1469,9327,0.0909,-0.1429,-0.1384,0.1943
1,2000,Total,Bedsitter,365.0,89,382,0.0429,-0.2843,0.0349,0.9778
2,2000,Total,1 Bedroom,540.0,741,4063,0.1020,-0.1692,-0.1136,-0.0326
3,2000,Total,Not Specified,445.0,34,511,-0.1524,-0.3904,-0.3200,0.2593
4,2000,Total,2 Bedrooms,750.0,517,3741,0.1194,-0.1979,-0.2083,0.6056
5,2000,Total,3 Bedrooms,1000.0,74,575,0.1111,-0.2780,0.0137,0.0882
6,2000,Total,4 or more Bedrooms,1675.0,s,55,0.6919,-0.0429,1.0000,6.0000
7,2000,House,Total,710.0,s,184,0.0965,-0.1647,-0.3182,0.7647
8,2000,House,Bedsitter,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2000,House,1 Bedroom,500.0,s,55,0.0000,-0.2187,-0.2143,0.3750


In [11]:
# Check df shape and null values
print(r135.shape)
print(r135.isnull().sum())

(16397, 10)
postcode                  0
dwelling_type             0
bed_number                0
median_rent_newb      12274
new_bonds_no          12274
total_bonds_no         6095
Qdelta_median_rent    12276
Adelta_median_rent    12277
Qdelta_new_bonds      12276
Adelta_new_bonds      12277
dtype: int64


**NOTE:**

Note that an alarming 3/4 of the data has null values. This is because the data is broken down to very granular level - first by dwelling type (Total, house, townhouse, flat/unit, other) and then by bed_numbers (see below cell).

In [12]:
print(r135.groupby('dwelling_type').size(),'\n')
print(r135.groupby('bed_number').size())

dwelling_type
Flat/Unit    3160
House        3653
Other        3257
Total        3903
Townhouse    2424
dtype: int64 

bed_number
1 Bedroom             2336
2 Bedrooms            2655
3 Bedrooms            2637
4 or more Bedrooms    2330
Bedsitter             1296
Not Specified         2310
Total                 2833
dtype: int64


**AGGREGATION:**

Given the amount of na present, we'll **aggregate the data to the total level of each postcode** (i.e. only consider the total number of bonds without classifying them into dwelling types nor bedroom numbers) to avoid excessive imputation. Although we'll inevitably lose the richness of information, the main function of the rent data is to serve as an input variable in the prediction of sales, where this level of granularity is not strictly necessary.

In [13]:
# Aggregate dwelling type and bed number, save as new df r135_ag

r135_ag = r135.loc[(r135['bed_number']=='Total') & (r135['dwelling_type']=='Total')]
r135_ag = r135_ag.drop(columns=['bed_number','dwelling_type'], axis=1) # Drop bed_number and dwelling_type

print(r135_ag.shape)
print(r135_ag.isnull().sum())

(617, 8)
postcode                0
median_rent_newb      181
new_bonds_no          181
total_bonds_no         51
Qdelta_median_rent    181
Adelta_median_rent    181
Qdelta_new_bonds      181
Adelta_new_bonds      181
dtype: int64


By aggregating the data, we're able to bring down the proportion of na from 3/4 to around 1/3. But there's still need for imputation. According to the data interpretation note:

<em><b>"For confidentiality, we don't report rents in any geographical area where the number of new bonds is 10 or less (shown as na). Statistics calculated from sample sizes between 10 an 30 are shown by an 's' in the relevant table"</b></em>

In [14]:
print("number of 's' in new_bonds_no:", r135_ag.loc[r135_ag['new_bonds_no']=='s'].shape[0])
print("number of 's' in total_bonds_noA:", r135_ag.loc[r135_ag['total_bonds_no']=='s'].shape[0],"\n")
print("number of na in new_bonds_no:", r135_ag.loc[r135_ag['new_bonds_no'].isnull()].shape[0])
print("number of na in total_bonds_no:", r135_ag.loc[r135_ag['total_bonds_no'].isnull()].shape[0])

number of 's' in new_bonds_no: 88
number of 's' in total_bonds_noA: 47 

number of na in new_bonds_no: 181
number of na in total_bonds_no: 51


<b>IMPUTATION</b>
* For 'new_bonds_no' and 'total_bonds_no' columns:
    * Impute na with 5
    * Impute s with 20
    
* For 'median_rent_newb' column
    * Impute na with median of rents of all POAs

In [15]:
# Impute 's' in 'new_bonds_no' and 'total_bonds_no' with 20
r135_ag.loc[r135_ag['new_bonds_no']=='s','new_bonds_no'] = 20.0
r135_ag.loc[r135_ag['total_bonds_no']=='s', 'total_bonds_no'] = 20.0

# Impute na in 'new_bonds_no' and 'total_bonds_no' with 5
r135_ag.loc[r135_ag['new_bonds_no'].isnull(),'new_bonds_no'] = 5.0
r135_ag.loc[r135_ag['total_bonds_no'].isnull(), 'total_bonds_no'] = 5.0

# Cast both variables as float (was object)
r135_ag['new_bonds_no'] = r135_ag['new_bonds_no'].astype(float)
r135_ag['total_bonds_no'] = r135_ag['total_bonds_no'].astype(float)

In [16]:
# Impute na in 'median_rent' with median of the column
r135_ag['median_rent_newb'].fillna(r135_ag['median_rent_newb'].median(), inplace=True)

# Check na in the df again
print(r135_ag.isnull().sum())

postcode                0
median_rent_newb        0
new_bonds_no            0
total_bonds_no          0
Qdelta_median_rent    181
Adelta_median_rent    181
Qdelta_new_bonds      181
Adelta_new_bonds      181
dtype: int64


In [17]:
# Prepare for merging
r135_ag['key_r'] = 'r135' # Add key

# Set postcode as index
r135_ag = r135_ag.set_index('postcode')
r135_ag.head(1)

,median_rent_newb,new_bonds_no,total_bonds_no,Qdelta_median_rent,Adelta_median_rent,Qdelta_new_bonds,Adelta_new_bonds,key_r
postcode,,,,,,,,
2000,600.0,1469.0,9327.0,0.0909,-0.1429,-0.1384,0.1943,r135


### End of the rent cleaning process ##
----



# STEP3. JOIN SALES AND RENT DATA OF THE SAME QUARTER #

### !! Please note !! ###
**A. the issue number of the sales and rent table of the same quarter are different**
<br>For example, for Q1 2021, the issue number of the sales table is 136 and that of the rent table is 135. The resulting data frames are s136 and r135_ag.</br>

**B. the sales table and the rent table contains different numbers of postcodes**
<br>For example, s136 (sales table) contains 544 unique postcodes while r135 (rent table) contains 617. We will merge the rent table into the sales table, i.e. only keep postcodes that are available in the sales table </br>

In [18]:
q121 = s136.join(r135_ag, on='postcode')
q121.head(1)

,postcode,dwelling_type,median_price,mean_price,sales_no,Qdelta_median_price,Adelta_median_price,Qdelta_sales_no,Adelta_sales_no,key_s,...,year,quarter,median_rent_newb,new_bonds_no,total_bonds_no,Qdelta_median_rent,Adelta_median_rent,Qdelta_new_bonds,Adelta_new_bonds,key_r
0,2000,Total,1371.0,2794.0,184.0,0.1425,0.0711,0.0888,0.5862,s136,...,2021,1,600.0,1469.0,9327.0,0.0909,-0.1429,-0.1384,0.1943,r135


In [19]:
q121['postcode'].nunique()

544

# STEP4 REPEAT ABOVE PROCESS TO ALL QUARTERS #

# STEP5 MERGE ALL QUARTERS INTO THE MASTER DF #

# STEP 6 MERGE FEATURES IN THE THE MASTER DF #